In [5]:
from pathlib import Path
import os
import numpy as np
import torch
from captum.attr import Saliency
import time
import sys

sys.path.append(os.path.abspath("../utils"))
import myextensions

In [6]:
torch.manual_seed(123)
np.random.seed(123)

model = myextensions.get_vgg()

xai = Saliency(model)

# DATASET_PATH = "../../dataset/filtered_correct_vgg16/"
# MODE = "no-attack"

# SAVE_DIR_PATH = f"/home/cat/uni/bakis/outputs/saliency/{MODE}"

# Path(f"{SAVE_DIR_PATH}/pos").mkdir(parents=True, exist_ok=True)
# Path(f"{SAVE_DIR_PATH}/neg").mkdir(parents=True, exist_ok=True)
# Path(f"{SAVE_DIR_PATH}/all").mkdir(parents=True, exist_ok=True)
# Path(f"{SAVE_DIR_PATH}/blended").mkdir(parents=True, exist_ok=True)


In [7]:
def save_3_heatmaps(dATASET_PATH, filename, attributions, input_image):
    filepath_pos = os.path.join(dATASET_PATH, "pos", filename)
    filepath_neg = os.path.join(dATASET_PATH, "neg", filename)
    filepath_all = os.path.join(dATASET_PATH, "all", filename)
    filepath_blended = os.path.join(dATASET_PATH, "blended", filename)

    # myextensions.save_result(attributions= attributions, image= input_image,
    #              save_path=filepath_pos, method="heat_map", sign="positive", outlier_perc=2)
    
    # myextensions.save_result(attributions= attributions, image= input_image,
    #             save_path=filepath_neg, method="heat_map", sign="negative", outlier_perc=2)
    
    myextensions.save_result(attributions= attributions, image= input_image,
                save_path=filepath_all, method="heat_map", sign="all", outlier_perc=2)
        
    myextensions.save_result(attributions= attributions, image= input_image,
                save_path=filepath_blended, method="blended_heat_map", sign="all", outlier_perc=2)
    

def print_time(savepath,timings, filenum):
    timings_np = np.array(timings)
    with open(f'{savepath}/time_statistics{filenum}.txt', "w") as f:
        f.write("=== Attribution Time Stats ===\n")
        f.write(f"Total images:      {len(timings)}\n")
        f.write(f"Average time:      {timings_np.mean():.4f} s\n")
        f.write(f"Standard deviation:{timings_np.std():.4f} s\n")
        f.write(f"Minimum time:      {timings_np.min():.4f} s\n")
        f.write(f"Maximum time:      {timings_np.max():.4f} s\n")

In [8]:
timings = []

MODES = \
[
    "adversial-patch/success", "adversial-patch/fail",
    "feature-adversaries/success", "feature-adversaries/fail",
    "fgsm/success", "fgsm/fail",
    "no-attack", 
    "pgd/success", "pgd/fail",
    "shadow-attack-nontargeted/success", "shadow-attack-nontargeted/fail",
    "square-attack-l2/success", "square-attack-l2/fail",
    "square-attack-linf/success", "square-attack-linf/fail",
]

for mode in MODES:
    dATASET_PATH = f"../../inputs/{mode}"
    sAVE_DIR_PATH = f"/home/cat/uni/bakis/outputs/saliency/{mode}"

    Path(f"{sAVE_DIR_PATH}/pos").mkdir(parents=True, exist_ok=True)
    Path(f"{sAVE_DIR_PATH}/neg").mkdir(parents=True, exist_ok=True)
    Path(f"{sAVE_DIR_PATH}/all").mkdir(parents=True, exist_ok=True)
    Path(f"{sAVE_DIR_PATH}/blended").mkdir(parents=True, exist_ok=True)

    sorted_filenames = os.listdir(dATASET_PATH)
    sorted_filenames.sort()

    for filename in sorted_filenames:
        full_path = os.path.join(dATASET_PATH, filename)

        input_image, _ = myextensions.get_image(full_path, myextensions.PREPROCESS_ATTACK)

        pred_label_idx, predicted_label, prediction_score = myextensions.get_prediction(model, input_image, topk = 5)

        input_image.requires_grad_()

        start_time = time.perf_counter()

        attribution = xai.attribute(input_image, target=pred_label_idx[0]) \
                    + xai.attribute(input_image, target=pred_label_idx[1]) \
                    + xai.attribute(input_image, target=pred_label_idx[2]) \
                    + xai.attribute(input_image, target=pred_label_idx[3]) \
                    + xai.attribute(input_image, target=pred_label_idx[4]) \

        elapsed = time.perf_counter() - start_time
        
        timings.append(elapsed)

        try:
            save_3_heatmaps(sAVE_DIR_PATH, filename, attribution, input_image)
        except Exception:
            print(f"Exception with: {full_path}")
            pass


    if (timings != []):
        print_time(sAVE_DIR_PATH ,timings, 0)
        timings = []